In [ ]:
#|default_exp migrate

# Migrate to nbprocess
> Utilities for migrating to nbprocess

In [ ]:
#|export
from nbprocess.process import first_code_ln
from nbprocess.processors import nb_fmdict, construct_fm, insert_frontmatter, is_frontmatter
from nbprocess.read import read_nb, config_key
from nbprocess.sync import write_nb
from nbprocess.clean import process_write, wrapio
from fastcore.all import *
from json import loads

## Convert fastpages blog posts to nbprocess

In [ ]:
#|export
def _get_fm(path): return nb_fmdict(read_nb(path), remove=False)
def _get_raw_fm(nb): 
    return first(nb.cells.filter(lambda x: x.cell_type == 'raw')).source

In [ ]:
#|export
def _cat_slug(fmdict):
    "Get the partial slug from the category front matter."
    cats = [c for c in fmdict.get('categories', '').strip().strip('][').split(', ') if c]
    return '/' + '/'.join(sorted(cats)) if cats else ''

In [ ]:
#|hide
_fm1 = _get_fm('../tests/2020-09-01-fastcore.ipynb')
test_eq(_cat_slug(_fm1), '/fastai/fastcore')

_fm2 = _get_fm('../tests/2020-02-20-test.ipynb')
test_eq(_cat_slug(_fm2), '/jupyter')

In [ ]:
#|export
def _file_slug(fname): 
    "Get the partial slug from the filename."
    p = Path(fname)
    dt = '/'+p.name[:10].replace('-', '/')+'/'
    return dt + p.stem[11:]    

In [ ]:
#|hide
test_eq(_file_slug('../tests/2020-09-01-fastcore.ipynb'), 
        '/2020/09/01/fastcore')

In [ ]:
#|export
def _add_alias(fm:dict, path:Path):
    if 'permalink' in fm: fm['aliases'] = '[' + fm.pop('permalink').strip() + ']'
    else: fm['aliases'] = '[' + _cat_slug(fm) + _file_slug(path) + ']'

### Migrate notebooks

In [ ]:
#|export
def migrate_nb_fm(path, overwrite=True):
    "Migrate fastpages front matter in notebooks to a raw cell."
    nb = read_nb(path)
    if is_frontmatter(nb): return None
    fm = nb_fmdict(nb)
    _add_alias(fm, path)
    insert_frontmatter(nb, fm_dict=fm)
    if overwrite: write_nb(nb, path)
    return nb

In [ ]:
_nb = migrate_nb_fm('../tests/2020-09-01-fastcore.ipynb', overwrite=False)
print(_get_raw_fm(_nb))

---
title: "fastcore: An Underrated Python Library"
description: A unique python library that extends the python programming language and provides utilities that enhance productivity.
author:  "<a href='https://twitter.com/HamelHusain'>Hamel Husain</a>"
image:  images/copied_from_nb/fastcore_imgs/td.png
categories:  [fastcore, fastai]
aliases: [/fastcore/]
---


In [ ]:
_nb = migrate_nb_fm('../tests/2020-02-20-test.ipynb', overwrite=False)
print(_get_raw_fm(_nb))

---
title: Fastpages Notebook Blog Post
description: A tutorial of fastpages for Jupyter notebooks.
image:  images/chart-preview.png
categories:  [jupyter]
aliases: [/jupyter/2020/02/20/test]
---


### Migrate Markdown Files

In [ ]:
#|export

_re_fm_md = re.compile(r'^---(.*\S+.)?---', flags=re.DOTALL)

def _md_fmdict(txt):
    "Get front matter as a dict from a markdown file."
    m = _re_fm_md.match(txt)
    if m:
        fm = [s.split(':', 1) for s in m.group(1).splitlines() if s]
        return {k:v.strip() for k,v in fm if k and v}
    else: return {}

In [ ]:
#|hide
_mdtxt = Path('../tests/2020-01-14-test-markdown-post.md').read_text()
_res = _md_fmdict(_mdtxt)
_res

{'toc': 'true',
 'layout': 'post',
 'description': 'A minimal example of using markdown with fastpages.',
 'categories': '[markdown]',
 'title': 'An Example Markdown Post'}

In [ ]:
#|hide
test_eq(_res, {'toc': 'true',
               'layout': 'post',
               'description': 'A minimal example of using markdown with fastpages.',
               'categories': '[markdown]',
               'title': 'An Example Markdown Post'})

In [ ]:
#|export
def migrate_md_fm(path, overwrite=True):
    "Make fastpages front matter in markdown files quarto compliant."
    p = Path(path)
    md = p.read_text()
    fm = _md_fmdict(md)
    if fm:
        _add_alias(fm, path)
        txt = _re_fm_md.sub(construct_fm(fm), md)
        if overwrite: p.write_text(txt)
        return txt
    else: return md 

Here is what the front matter of a markdown post looks like before:

In [ ]:
#|eval: false
print(run('head -n13 ../tests/2020-01-14-test-markdown-post.md'))

---
toc: true
layout: post
description: A minimal example of using markdown with fastpages.
categories: [markdown]
title: An Example Markdown Post
---
# Example Markdown Post

## Basic setup

Jekyll requires blog post files to be named according to the following format:


And this is what it looks like after:

In [ ]:
_res = migrate_md_fm('../tests/2020-01-14-test-markdown-post.md', overwrite=False)
print(_res[:300])

---
title: An Example Markdown Post
description: A minimal example of using markdown with fastpages.
categories: [markdown]
aliases: [/markdown/2020/01/14/test-markdown-post]
---
# Example Markdown Post

## Basic setup

Jekyll requires blog post files to be named according to the following format:




In [ ]:
#|hide
assert """---
title: An Example Markdown Post
description: A minimal example of using markdown with fastpages.
categories: [markdown]
aliases: [/markdown/2020/01/14/test-markdown-post]
---""" in _res

## Convert nbdev v1 projects to nbdev v2

### Migrate Directives

In [ ]:
#|export
def _re_v1():
    d = ['default_exp', 'export', 'exports', 'exporti', 'hide', 'hide_input', 'collapse_show', 
         'collapse_hide', 'hide_output', 'collapse_input', 'collapse_output', 'default_cls_lvl']
    d += L(config_key('tst_flags', [], path=False, missing_ok=True))
    d += [s.replace('_', '-') for s in d] # allow for hyphenated version of old directives
    _tmp = '|'.join(list(set(d)))
    return re.compile(f"^[ \f\v\t]*?(#)\s*({_tmp})", re.MULTILINE)

def _repl_directives(code_str): 
    def _fmt(x): return f"#|{x.group(2).replace('-', '_')}"
    return _re_v1().sub(_fmt, code_str)


In [ ]:
#|hide
_test_dir = """
#default_exp
 #export
# collapse-show
#collapse-hide
#collapse-output
# collapse_output
not_dir='#export'
# hide_input
foo
"""
test_eq(_repl_directives(_test_dir), "\n#|default_exp\n#|export\n#|collapse_show\n#|collapse_hide\n#|collapse_output\n#|collapse_output\nnot_dir='#export'\n#|hide_input\nfoo\n")

In [ ]:
#|export
def _repl_v1dir(nb):
    "Replace nbdev v1 with v2 directives."
    for cell in nb['cells']:
        if cell.get('source') and cell.get('cell_type') == 'code':
            ss = cell['source'].copy()
            first_code = first_code_ln(ss, re_pattern=_re_v1())
            if not first_code: first_code = len(ss)
            if not ss: pass
            else: cell['source'] = [_repl_directives(c) for c in ss[:first_code]] + ss[first_code:]

In [ ]:
#|hide
_code = _test_dir.splitlines(True)

tst = {'cell_type': 'code', 'execution_count': 26,
       'metadata': {'hide_input': True, 'meta': 23},
       'outputs': [{'execution_count': 2,
                    'data': {
                        'application/vnd.google.colaboratory.intrinsic+json': {'type': 'string'},
                        'plain/text': ['sample output',]
                    }, 'output': 'super'}],
       'source': _code}
nb = {'metadata': {'kernelspec': 'some_spec', 'jekyll': 'some_meta', 'meta': 37}, 'cells': [tst]}

_repl_v1dir(nb)
test_eq(nb['cells'][0], {'cell_type': 'code',
     'execution_count': 26,
     'metadata': {'hide_input': True, 'meta': 23},
     'outputs': [{'execution_count': 2,
     'data': {'application/vnd.google.colaboratory.intrinsic+json': {'type': 'string'},
     'plain/text': ['sample output']},
     'output': 'super'}],
     'source': ['\n', '#|default_exp\n', '#|export\n', '#|collapse_show\n', '#|collapse_hide\n', '#|collapse_output\n', '#|collapse_output\n', "not_dir='#export'\n", '# hide_input\n', 'foo\n']
    })

In [ ]:
#|export
@call_parse
def nbprocess_migrate_directives(
    fname:str=None, # A notebook name or glob to convert
    disp:bool=False,  # Print the outputs with newly formatted directives
    stdin:bool=False, # Read input stream and not nb folder
    no_skip:bool=False, # Do not skip directories beginning with an underscore
):
    "Convert all directives in `fname` from v1 to v2."
    _write = partial(_process_write, warn_msg='Failed to replace directives', proc_nb=_repl_v1dir)
    if stdin: _write(f_in=_wrapio(sys.stdin), f_out=_wrapio(sys.stdout))
    _skip_re = None if no_skip else '^[_.]'
    if fname is None: fname = config_key("nbs_path", '.', missing_ok=True)
    for f in globtastic(fname, file_glob='*.ipynb', skip_folder_re=_skip_re): _write(f_in=f, disp=disp)
    

## Export -

In [ ]:
#|hide
#|eval:false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()